In [18]:
import requests
from bs4 import BeautifulSoup
import dataset
import time

#### Mining main page containing the diseases

In [3]:
# Retrieve the main page with all diseases
main_page = requests.get('https://www.dermis.net/dermisroot/es/list/all/search.htm')
# Parse into a BS object
soup = BeautifulSoup(main_page.text,'html.parser')

In [15]:
# Get all diseases in a list
all_diseases = soup.find_all(class_='list')

# List for disease codes
codes = []
for disease in all_diseases:
    codes.append(disease.get('href').split('/')[-2])

languages = ['tr','jp','pt','es','fr','de','en']



#### Mining individual disease pages

In [51]:
def scrape_page(page_id,language):
    """
    Function to scrape individual pages from DermIS
    Inputs:
        -page_id: The ID of the disease
        -language: Language of interest
    Output: Dictionary with:
        -disease_name: Name of the disease (string)
        -disease_definition: Definition of the disease (string)
        -image_codes: Codes of the disease images (list of integers)
    """
    
    
    url_part1 = 'https://www.dermis.net/dermisroot/'
    url_part2 = '/diagnose.htm'
    
    # Download de page source
    disease_page = requests.get(url_part1 + language + '/' + str(page_id) + url_part2)
    
    # Parse to BS
    soup_disease = BeautifulSoup(disease_page.text,'html.parser')
    
    # Parse information
    disease_definition = soup_disease.find_all('p',attrs={'class': None})[0].text
    disease_name = soup_disease.find(class_='Container',attrs={'id': 'ctl00_Main_pnlMain'}).find('h2').text
    images_links = soup_disease.find(class_='Container',attrs={'id': 'ctl00_Main_pnlMain'}).find_all('a')
    
    # Parse images
    image_codes = []
    for image in images_links:
        image_codes.append(image.get('href').split('/')[-2])
    
    result = {'disease_name': disease_name,
              'disease_definition': disease_definition,
              'image_codes': image_codes}
    
    return(result)

#### Iterating over pages and storing in SQLite database

In [ ]:
db = dataset.connect('sqlite::///dermis.db')
table = db['scrapped']

for disease_code in codes:
    for language in languages:
        tmp = scrape_page(disease_code, language)
        tmp['language'] = language
        table.insert(tmp)
        time.sleep(10)
        